# Starting the Game Model


## Combinação de Modelos
- Modelo 1 - Random Forest Classifier (ALL)

Se não encontrar WIN, vai pro próximo
- Modelo 2 - Random Forest Classifier (col by col)

Se Não encontrar WIN, vai pro próximo
- Model 3 - Rede Neural

Se Não encontrar, vamos apelar para uma jogada aleatória

- Modelo 4 - Aleatório (Aqui excluímos as piores posições [2,4,6,8] ) e fazemos o soteio


### Lógica a busca pela vitória
exemplo:

Turno 1 - Meu turno

(1_,'_','_','_','_','_','_','_','_')  Joguei

Turno 2 - Máquina

Ela irá jogar o próximo valor em todas posições possíveis não ocupadas, exemplo

(1_,'2','_','_','_','_','_','_','_') = 0 

(1_,'_','2','_','_','_','_','_','_') = 0 
.
.
.
(1_,'_','_','_','_','_','_','_','2') = 0 

Então irá examinar... se ocorrer vitória (1) ou empate (0.5), ela joga nesta posição o 2, caso contrário, ela testa outro número


Caso ele não encontre 1 ou 0.5... ele joga um valor aleatório removendo os valores
[2,4,6,8] dos valores LIVRES... pois são localizações ruins

- ~Toda vez que ele check se há vitória, derrota ou empate... caso dê okay, ele faz o treinamento da rede neural com novos valores inputados!~ (fazia na rede neural


## OBS: Lances indefinidos quando resultam em Win

é perigoso, pois o modelo não tem muitos dados para analisar ainda... logo ele faz um chute, então que seja um chute bom... por isso, utilize a função "sortear_pos_boa()", pois pelo menos depois da segunda rodada, a máquina pode matar o jogo

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from sklearn.ensemble import RandomForestClassifier

import random
import pickle

### Random Forest Classifier MODELS

##### Load <span style="color:red">Model</span>

In [2]:
# Random Forest Classifier All
model_all = pickle.load(open('../02_train/save_model/random_forest_classifier_all.sav','rb'))

In [3]:
test = [
    [1,3,5,0,0,0.,2,4,0], # Win
    [1,3,0,5,0,0,2,4,6], # Lost
    [6,1,2,9,3,4,5,8,7], # Draw
    [2,4,0,0,0,0,1,3,5], # Win
    [1,3,0,0,5,0,2,4,6], # Lost
    [1,3,0,5,0,6,2,4,0], # Vou vencer na pŕoxima
    [1,0,0,0,0,0,0,0,0], # Indefinido (Perigoso dar Vitória.. isso não é bom)
    [2,0,3,0,0,0,0,1,0], # Indefinido (Perigoso dar Vitória.. isso não é bom)
]
test = np.array(test)
test.shape
print(model_all.predict(test))

['win' 'lost' 'draw' 'win' 'lost' 'lost' 'win' 'win']


##### Load <span style="color:red">Model</span>

In [4]:
# Random Forest Classifier (por Coluna)
model123 = pickle.load(open('../02_train/save_model/random_forest_classifier_123.sav','rb'))
model456 = pickle.load(open('../02_train/save_model/random_forest_classifier_456.sav','rb'))
model789 = pickle.load(open('../02_train/save_model/random_forest_classifier_789.sav','rb'))
model147 = pickle.load(open('../02_train/save_model/random_forest_classifier_147.sav','rb'))
model258 = pickle.load(open('../02_train/save_model/random_forest_classifier_258.sav','rb'))
model369 = pickle.load(open('../02_train/save_model/random_forest_classifier_369.sav','rb'))
model357 = pickle.load(open('../02_train/save_model/random_forest_classifier_357.sav','rb'))
model159 = pickle.load(open('../02_train/save_model/random_forest_classifier_159.sav','rb'))

In [5]:
test = [
    [1,3,5], # Win
    [2,4,6], # Lost
]

test = np.array(test)
test.shape
print(model456.predict(test))
print(model159.predict(test))

['win' 'lost']
['win' 'lost']


### Rede Neural - Model

##### Load <span style="color:red">Model</span>

In [6]:
model_rede = tf.keras.models.load_model('../02_train/save_model/model.h5')

##### Load <span style="color:red">WEIGHT</span>

In [7]:
model_rede.load_weights('../02_train/save_weight/weight.h5')

In [8]:
test = [
    [1,3,5,0,0,0.,2,4,0], # Win
    [1,3,0,5,0,0,2,4,6], # Lost
    [6,1,2,9,3,4,5,8,7], # Draw
    [2,4,0,0,0,0,1,3,5], # Win
    [1,3,0,0,5,0,2,4,6], # Lost
    [1,3,0,5,0,6,2,4,0], # Vou vencer na pŕoxima
    [0,0,0,0,0,0,0,1,0], # Indefinido (Perigoso dar Vitória.. isso não é bom)
    [2,0,3,0,0,0,0,1,0], # Indefinido (Perigoso dar Vitória.. isso não é bom)
    
]
test = pd.DataFrame(test)
test.shape

model_rede.predict(test)

array([[1.       ],
       [0.       ],
       [0.7825757],
       [1.       ],
       [0.       ],
       [0.       ],
       [1.       ],
       [1.       ]], dtype=float32)

### Funções

In [9]:
# Função para transformar a saída do modelo Neural pra 'win','lost' ou'draw'
def f(x):
    val = x[0]
    #val = x
    if val >= 0.79:
        return 'win'
    if 0.40< val and val < 0.79:
        return 'draw'
    if val <= 0.40:
        return 'lost'

In [10]:
#pred_df['pred'] = pred_df[['pred']].apply(f,axis=1)

In [11]:
# creating the table
def df():
    col = []
    for n in range(1,10):
        col.append(n)
    col.append('resultado')

    df = pd.DataFrame(columns=col)
    return df

In [12]:
def n(x): # even odd function
    
    if x == 0:
        return 0.5 # para pular msm...
    
    elif x%2 == 0: # Número par retorna 0
        return x%2
    
    else:# número impar retorna 1
        return x%2

#even_odd(2)

In [13]:
# é utilizado para pegar valores de um vetor para os modelos por coluna

# def sortear_pos_model_col(self): Utilizado aqui

def get_col(vector, vector_3_col):
    new_vector = []
    for i in vector_3_col:
        new_vector.append(vector[i-1])
    return new_vector

"""x = [10,20,30,40,50,60,70,80,90]
x_model = [1,5,4]
# quero pegar o indice 1 , 2 e 9
x_new = get_col(x,x_model)
x_new""";

## GAME

In [33]:
class ticTacToe():
    
    def __init__(self):
        
        self.df = df()
        self.pos ={
                    1:0,
                    2:0,
                    3:0,
                    4:0,
                    5:0,
                    6:0,
                    7:0,
                    8:0,
                    9:0,
                    'resultado':'unknown' }
        self.t = 1 # turnos
        
    
    #######################################################
    ################## RESTART GAME  ######################
    #######################################################
    def restart_df(self):
        self.df = df()
        
    
    def restart_game(self):
        self.pos ={
                    1:0,
                    2:0,
                    3:0,
                    4:0,
                    5:0,
                    6:0,
                    7:0,
                    8:0,
                    9:0,
                    'resultado':'unknown' }
        self.t = 1 # turnos
    #######################################################
        
        
    #######################################################
    ##################### TABLES ##########################
    #######################################################
    # Criar a Tabelas e dar appends nela
    def add_value_df(self):
        
        df_receber = pd.DataFrame( columns=list(self.pos.keys()) )
        for i in df_receber.columns:
            if i == 'resultado':
                df_receber[i] = [self.pos[i]]
            else:    
                df_receber[i] = [self.pos[int(i)]]
                
        self.df = self.df.append(df_receber,ignore_index=True)
    #########################################################

        
        
    #######################################################
    ##################### CHECKS ##########################
    #######################################################
    def check_defeat(self): # player é par
        if n(self.pos[1]) == n(self.pos[2]) == n(self.pos[3]) == 0:
            return 'break'
        if n(self.pos[4]) == n(self.pos[5]) == n(self.pos[6]) == 0:
            return 'break'
        if n(self.pos[1]) == n(self.pos[4]) == n(self.pos[7]) == 0:
            return 'break'
        if n(self.pos[1]) == n(self.pos[5]) == n(self.pos[9]) == 0:
            return 'break'
        if n(self.pos[2]) == n(self.pos[5]) == n(self.pos[8]) == 0:
            return 'break'
        if n(self.pos[3]) == n(self.pos[5]) == n(self.pos[7]) == 0:
            return 'break'
        if n(self.pos[3]) == n(self.pos[6]) == n(self.pos[9]) == 0:
            return 'break'
        if n(self.pos[7]) == n(self.pos[8]) == n(self.pos[9]) == 0:
            return 'break'
        return 'continue'

    def check_victory(self): # pc é impar
        if n(self.pos[1]) == n(self.pos[2]) == n(self.pos[3]) == 1:
            return 'break'
        if n(self.pos[4]) == n(self.pos[5]) == n(self.pos[6]) == 1:
            return 'break'
        if n(self.pos[1]) == n(self.pos[4]) == n(self.pos[7]) == 1:
            return 'break'
        if n(self.pos[1]) == n(self.pos[5]) == n(self.pos[9]) == 1:
            return 'break'
        if n(self.pos[2]) == n(self.pos[5]) == n(self.pos[8]) == 1:
            return 'break'
        if n(self.pos[3]) == n(self.pos[5]) == n(self.pos[7]) == 1:
            return 'break'
        if n(self.pos[3]) == n(self.pos[6]) == n(self.pos[9]) == 1:
            return 'break'
        if n(self.pos[7]) == n(self.pos[8]) == n(self.pos[9]) == 1:
            return 'break'
        return 'continue'
    
    def check_pos_full(self): # EMPATE
        if (self.pos[1] != 0 
            and self.pos[2] != 0 
            and self.pos[3] != 0 
            and self.pos[4] != 0
            and self.pos[5] != 0
            and self.pos[6] != 0
            and self.pos[7] != 0
            and self.pos[8] != 0
            and self.pos[9] != 0):
            
            return 'break'
    
        return 'continue'
    
    # Agora os turnos importam, pois turno impar é pc e turno par é player
    def check_turn(self):
        if self.t == 10: # Verificar tempo/turnos (f)
            return 'break'
        return 'continue'
    
    # Chamo esse método para verificar tudo
    def check(self):

        
        self.t +=1
        
        # Verificar derrota
        if self.check_defeat() == 'break':
            #print('derrotado')
            self.pos['resultado'] = 'lost'
            
            return 'break'
        
        # Verificar Vitoria
        if self.check_victory() == 'break':
            #print('vencedor')
            self.pos['resultado'] = 'win'
            
            return 'break'
        
        # Verificar Turno
        
        if self.check_turn() == 'break':
            #print('turno')
            self.pos['resultado'] = 'draw' # Empate
            
            return 'break'
        
        # Verificar Tudo
        if self.check_pos_full() == 'break':
            #print('tudo')
            self.pos['resultado'] = 'draw' # Empate
            
            return 'break'
        
        return 'continue'
    #######################################################



    #######################################################
    ################ TIPO DE JOGADAS  #####################
    #######################################################
    # Aqui eu set os numeros no braço usando INPUTS
    def player(self, player):
        if player == 'player':
            move = int(input('player: '))
            if self.pos[move] == 0:
                self.pos[move] = int(self.t)
            else:
                while True:
                    move = int(input('player, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t)
                        break
                        
        if player == 'pc':
            move = int(input('pc: '))
            if self.pos[move] == 0:
                self.pos[move] = int(self.t)

            else:
                while True:
                    move = int(input('pc, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t)
                        break
    
    # Aqui eu uso para deixar para ser TREINADA
    def player_random(self, player):
        if player == 'player':
            move = self.sortear_pos_qualquer()
            if self.pos[move] == 0:
                self.pos[move] = int(self.t) # add o valor do turno 
            else:
                while True:
                    move = int(input('player, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t) # add o valor do turno
                        break
                        
        if player == 'pc':
            move = self.sortear_pos_qualquer()
            if self.pos[move] == 0:
                self.pos[move] = int(self.t) # add o valor do turno

            else:
                while True:
                    move = int(input('pc, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t) # add o valor do turno
                        break
                        

    # Modelo Aplicado
    def player_model(self,player):
        if player == 'player':
            move = self.sortear_pos_qualquer() # Sortear
            #move = int(input('player: ')) # Manual
            if self.pos[move] == 0:
                self.pos[move] = int(self.t)
            else:
                while True:
                    move = int(input('player, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t)
                        break
        
        
        if player == 'pc':
            
            move = self.modelos()
            #move = self.sortear_pos_model_col()
            #if self.pos[move] == 0:
            self.pos[move] = int(self.t)
            
    #######################################################
    
    
    
    
    
    #######################################################
    ################ SORTEIOS DE JOGADAS ##################
    #######################################################
    def sortear_pos_qualquer(self): # Ganhar tempo
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]

        return random.sample(livres,1)[0] # sortear 1 número dentro dos LIVRES

    def sortear_pos_boa(self):
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]

        
        # chuta um valor bom
        pos_ruins = [2,4,6,8] # Essas são posições ruins
        
        for ruin in pos_ruins:
            try:
                # Vamos remover as posições ruins dentro das Livres
                livres.remove(ruin)
                return random.sample(livres,1)[0]
            # não tem mais posição boa pra remover... então ele dá problema
            # joga a ruim msm...
            except ValueError:
                return random.sample(pos_ruins,1)[0]
                
    ########################################################
    
    ########################################################
    #################### ORDEM MODELOS #####################
    def modelos(self):
        
        # Um modelo Chama o outro em caso de Fracasso
        
        # Modelo 1
        return self.sortear_pos_model_rede(
                    # Modelo 2
                    self.sortear_pos_model_all(
                        # Modelo 3
                        self.sortear_pos_model_col(
                            # Sortear
                            self.sortear_pos_boa()
                        )
                    )
                )
    ##########################################################


    #######################################################
    ################ JOGADAS MODELO ##################
    #######################################################
    def sortear_pos_model_all(self,proximo_modelo): # Modelo 1 - Analisa todas colunas de uma vez
        
        # primeira jogada
        #if self.t ==1:
            #return sortear_pos_boa()
            #return 5
        
        # VALORES LIVRES
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]

        
        # pegar todos valoreS já jogados até o momento
        vetor = [list(self.pos.values())[:-1]] 

        # faz um loop com os valores livres
        # e coloca o valor em cada posição e roda o modelo
        for pos_livre in livres:
            vetor[0][pos_livre-1] = self.t

            
            pred = model_all.predict(vetor)
            #resultado = f(pred[0][0]) # Resultado (já transformado em 0 ;0.5 ou 1)
            
            
            resultado = pred[0]
            
            if resultado == 'win':# Se não for Derrota... já vale
                #print(resultado)
                return pos_livre
            
            vetor[0][pos_livre-1] = 0 #Voltar o valor vazio novamente para realizar o looping novamente
        
        
        # Se Esse modelo não der certo, vamos para o modelo 2 (col by col)
        return proximo_modelo
        
       
    def sortear_pos_model_col(self,proximo_modelo): # Modelo 2 - analisa só as colunas importantes
        
        # VALORES LIVRES
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]

        
        # pegar todos valoreS já jogados até o momento
        vetor = [list(self.pos.values())[:-1]] 

        # Os modelos estão todos colocados em um vetor
        models = [model123,model456,model789,model147,model258,model369,model357,model159]
        
        # para chamá-los um por um e testar
        for model in models:
            if model is model123:
                vetor_model =[1,2,3]
            if model is model456:
                vetor_model =[4,5,6]
            if model is model789:
                vetor_model =[7,8,9]
            if model is model147:
                vetor_model =[1,4,7]
            if model is model258:
                vetor_model =[2,5,8]
            if model is model369:
                vetor_model =[3,6,9]
            if model is model357:
                vetor_model =[3,5,7]
            if model is model159:
                vetor_model =[1,5,9]
                
        
            for pos_livre in livres:
                vetor[0][pos_livre-1] = self.t
                
                vetor_new = get_col(vetor[0],vetor_model)
                
                # pega somente 3 colunas
                pred = model.predict([vetor_new])
                resultado = pred[0]
                
                
                if resultado == 'win':# Se não for Derrota... já vale
                    
                    return pos_livre
                
                vetor[0][pos_livre-1] = 0 #Voltar o valor vazio novamente para realizar o looping novamente
        
        
        # Se nada der certo vamos sortear um 
        return proximo_modelo
    
    
    def sortear_pos_model_rede(self,proximo_modelo):
        # VALORES LIVRES
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]


        # pegar todos valoreS já jogados até o momento
        vetor = [list(self.pos.values())[:-1]] 


        for pos_livre in livres:
            vetor[0][pos_livre-1] = self.t


            pred = model_rede.predict(vetor)

            #resultado = f(pred[0][0]) # Resultado (já transformado em 0 ;0.5 ou 1)
            resultado = f(pred[0])
            
            #print(resultado)

            if resultado == 'win':# Se não for Derrota... já vale
                
                return pos_livre

            vetor[0][pos_livre-1] = 0 


        # Se nada der certo vamos sortear um valor bom
        # pelo menos
        return proximo_modelo

    
    ########################################################
    
    
    
    
        
    # RUN DE PROGRAM
    def run(self):
        
        while True:
            
            # Jogar Player
            #self.player('pc')
            #self.player_random('pc')
            self.player_model('pc')
             
            # Check -----------------
            if self.check() == 'break':
                break
            
            
            # Jogar PC
            #self.player('player')
            #self.player_random('player')
            self.player_model('player')
             
            # Check -----------------
            if self.check() == 'break':
                break
            #------------------------
        
        
        self.add_value_df()
        
        
   
  

In [66]:
game = ticTacToe()

In [67]:
for i in range(0,100):
    game.restart_game()
    game.run()
game.df


,1,2,3,4,5,6,7,8,9,resultado
0,1,3,2,4,5,6,7,8,9,win
1,1,3,5,0,0,0,0,2,4,win
2,1,3,4,5,6,2,7,0,0,win
3,1,2,3,4,5,7,8,9,6,draw
4,1,2,3,5,7,6,9,8,4,win
5,1,3,5,0,0,0,2,0,4,win
6,1,3,5,0,0,0,4,2,0,win
7,1,3,2,5,7,8,4,6,9,win
8,1,3,5,2,0,4,0,0,0,win
9,1,3,5,4,0,0,0,0,2,win


In [68]:
#df = game.df
game.df[game.df['resultado']=='win'].count()

1            81
2            81
3            81
4            81
5            81
6            81
7            81
8            81
9            81
resultado    81
dtype: int64

In [69]:
game.df[game.df['resultado']=='draw'].count()

1            6
2            6
3            6
4            6
5            6
6            6
7            6
8            6
9            6
resultado    6
dtype: int64

In [19]:
#game.df

In [20]:
# Save TABLE
#game.df.to_pickle('./dados.pkl')

In [21]:
#load_df = pd.read_pickle('./dados.pkl')

In [22]:
#load_df.drop_duplicates().reset_index().drop('index',axis=1)

In [23]:
##### Remover Valores duplicados
#dub = game.df.drop_duplicates().reset_index().drop('index',axis=1)
#dub.tail()